In [ ]:
from typing import List

from io_agent.plant.fighter import FighterEnv, fighter_env_params
from io_agent.trainer import ControlLoop, Transition
from io_agent.control.mpc import MPC
from io_agent.control.rmpc import RobustMPC


def run_mpc(env_length: int = 60,
            horizon: int = 20,
            use_foresight: bool = True
            ) -> List[Transition]:
    plant = FighterEnv(max_length=env_length, env_params=fighter_env_params)
    agent = MPC(env_params=fighter_env_params,
                horizon=horizon)
    trainer = ControlLoop(
        state_disturbance=plant.state_disturbance,
        output_disturbance=plant.output_disturbance,
        plant=plant,
        controller=agent
    )

    return trainer.simulate(
        initial_state=None,
        use_foresight=use_foresight,
    )




## Trajectory Cost Distributions

Run each experiment 10 times.


In [ ]:

mpc_obl_trajectories = [run_mpc(use_foresight=False) for _ in range(10)]
mpc_dst_trajectories = [run_mpc(use_foresight=True) for _ in range(10)]

Plot the cost distributions.

In [ ]:
from itertools import chain
import plotly.graph_objects as go
import plotly.figure_factory as ff


mpc_obl_costs = [transition.cost for transition in chain(*mpc_obl_trajectories)]
mpc_dst_costs = [transition.cost for transition in chain(*mpc_dst_trajectories)]

fig = go.FigureWidget()
fig.update_layout(
)
fig = ff.create_distplot([mpc_obl_costs, mpc_dst_costs],
                         group_labels=["MPC (obl)", "MPC (dst)"],
                         colors=["#333F44", "#37AA9C"],
                         bin_size=2, show_rug=False)

fig.update_layout(
    template="plotly_white",
    width=600,
    height=400,
    title=dict(text=f"MPC cost density with {len(mpc_dst_trajectories)} trials", x=0.5),
    yaxis=dict(title=dict(text="density")),
    xaxis=dict(title=dict(text="cost")),
    bargap=0.1
)
